# Création d'un dictionnaire à partir de fichiers csv

In [5]:
import csv
from datetime import datetime
import os
import glob

def create_timestamp(date):
    dt = datetime(int(date[0:4]), int(date[5:7]), int(date[8:10]), 
                   int(date[11:13]), int(date[14:16]), int(date[17:19]))
    
    dt = datetime.timestamp(dt)
    return dt


def filename_from_path(file):
    
    begin = len(file)-1
    
    while file[begin] != ".":
        begin -= 1
        
    end = begin
    
    while file[begin] != "\\":
          begin -= 1
          
    begin += 1
    return file[begin:end]
    
    

notebook_path = os.path.abspath("Notebook.ipynb")
chemin = os.path.join(os.path.dirname(notebook_path), "results/assets")

paths = glob.glob(chemin+'/*')

data = []

for i in range(0, 40):
    
    name = filename_from_path(paths[i])
    
    with open(paths[i], newline='') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            point = {}      
            point['asset_id'] = name
            point['recorded_at'] = create_timestamp(row['recorded_at'])        
            
            g = {}
            coord = []
            coord.append(float(row['longitude']))
            coord.append(float(row['latitude']))
            g['coordinates'] = coord
            g['type'] = "Point"
            
            loc = {}
            loc['geo'] = g
            point['location'] = loc
            
            data.append(point)
        
    data = sorted(data, key=lambda point: point['recorded_at'])   # tri selon recorded_at

    
print('done')

done


# Index géo spatial

In [6]:
import pymongo
from pymongo import GEOSPHERE

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

resp = collection.create_index([("location.geo", GEOSPHERE)])
print ("index response:", resp)

index response: location.geo_2dsphere


# Importation du dictionnaire dans la base de données 'fcd'

In [7]:
client = pymongo.MongoClient('localhost',27017)
mydb = client["fcd"]
collection = mydb["assets"]

print('insert')
collection.insert_many(data)
print('done')

insert
done
